### USER BASED FINANCIAL ANALYSIS

In [2]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
def load_data():
    # Load user data
    user_df = pd.read_excel("./dummy_user_data.xlsx")
    # Load financial rules
    financial_rules_df = pd.read_excel("Updated_Financial_Analytics.xlsx")
    return user_df, financial_rules_df
def calculate_budget(salary, marital_status, region, num_children, category_rules):
    # Extract relevant budget allocation percentages
    category_row = category_rules[(category_rules['Salary'] == salary) & (category_rules['Region'] == region) & (category_rules['Category'] == marital_status)]
    if category_row.empty:
        needs_percentage, wants_percentage, savings_percentage = 50, 30, 20
    else:
        needs_percentage, wants_percentage, savings_percentage = map(lambda x: sum(map(int, x.strip('%').split('-'))) / 2,
                                                                    category_row.iloc[0][['Needs (%)', 'Wants (%)', 'Savings (%)']])
    # Adjust based on number of children
    needs_percentage += num_children * 5
    wants_percentage -= num_children * 5
    # Adjust based on region
    if region == "Tier 1":
        needs_percentage += 5
        wants_percentage -= 5
    elif region == "Tier 4":
        needs_percentage -= 5
        wants_percentage += 5
    # Calculate financial allocations
    needs_amount = salary * (needs_percentage / 100)
    wants_amount = salary * (wants_percentage / 100)
    savings_amount = salary * (savings_percentage / 100)
    return needs_amount, wants_amount, savings_amount
def display_user_details(selected_user, user_df, category_rules):
    user_row = user_df[user_df['User Name'] == selected_user].iloc[0]
    salary = user_row['Salary']
    marital_status = user_row['Marital Status']
    region = user_row['Region']
    num_children = user_row.get('Children', 0)
    needs, wants, savings = calculate_budget(salary, marital_status, region, num_children, category_rules)
    output_df = pd.DataFrame({
        "User": [selected_user],
        "Salary": [salary],
        "Marital Status": [marital_status],
        "Region": [region],
        "Number of Children": [num_children],
        "Needs": [needs],
        "Wants": [wants],
        "Savings": [savings]
    })
    display(output_df)
def main():
    user_df, category_rules = load_data()
    user_dropdown = widgets.Dropdown(
        options=user_df['User Name'].unique(),
        description='Select User:',
        style={'description_width': 'initial'}
    )
    output = widgets.Output()
    def on_user_change(change):
        with output:
            output.clear_output()
            display_user_details(change.new, user_df, category_rules)
    user_dropdown.observe(on_user_change, names='value')
    display(user_dropdown, output)
main()


Dropdown(description='Select User:', options=('Lori Bailey', 'Kenneth Yu', 'Jose Phillips', 'Janice Davies', '…

Output()

### SALARY BASED ANALYSIS

In [3]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
def load_data():
    # Load financial rules
    financial_rules_df = pd.read_excel("./Updated_Financial_Analytics.xlsx")
    return financial_rules_df
def calculate_budget(salary, marital_status, region, num_children, category_rules):
    # Extract relevant budget allocation percentages
    category_row = category_rules[(category_rules['Region'] == region) & (category_rules['Category'] == marital_status)]
    if category_row.empty:
        needs_percentage, wants_percentage, savings_percentage = 50, 30, 20
    else:
        needs_percentage, wants_percentage, savings_percentage = map(lambda x: sum(map(int, x.strip('%').split('-'))) / 2,
                                                                    category_row.iloc[0][['Needs (%)', 'Wants (%)', 'Savings (%)']])
    # Adjust based on number of children
    if marital_status in ["Married", "Divorced"] and num_children:
        needs_percentage += num_children * 5
        wants_percentage -= num_children * 5
    # Adjust based on region
    if region == "Tier 1":
        needs_percentage += 5
        wants_percentage -= 5
    elif region == "Tier 4":
        needs_percentage -= 5
        wants_percentage += 5
    # Calculate financial allocations
    needs_amount = salary * (needs_percentage / 100)
    wants_amount = salary * (wants_percentage / 100)
    savings_amount = salary * (savings_percentage / 100)
    return needs_amount, wants_amount, savings_amount
def display_result(salary, marital_status, region, num_children, category_rules, output):
    needs, wants, savings = calculate_budget(salary, marital_status, region, num_children, category_rules)
    output_df = pd.DataFrame({
        "Salary": [salary],
        "Marital Status": [marital_status],
        "Region": [region],
        "Number of Children": [num_children],
        "Needs": [needs],
        "Wants": [wants],
        "Savings": [savings]
    })
    with output:
        output.clear_output()
        display(output_df)
def main():
    category_rules = load_data()
    # Inject CSS for pastel colors
    display(HTML("""
    <style>
        button[data-value="Single"] { background-color: #FFDDC1 !important; } /* Peach */
        button[data-value="Married"] { background-color: #C1E1C1 !important; } /* Pastel Green */
        button[data-value="Divorced"] { background-color: #ADD8E6 !important; } /* Soft Blue */
        button[data-value="Tier 1"] { background-color: #FFC8A2 !important; } /* Soft Orange */
        button[data-value="Tier 2"] { background-color: #D4A5A5 !important; } /* Dusty Rose */
        button[data-value="Tier 3"] { background-color: #B5EAD7 !important; } /* Light Teal */
        button[data-value="Tier 4"] { background-color: #A2D2FF !important; } /* Soft Sky Blue */
    </style>
    """))
    salary_input = widgets.FloatText(
        description="Enter Salary:",
        style={'description_width': 'initial'}
    )
    marital_status_buttons = widgets.ToggleButtons(
        options=["Single", "Married", "Divorced"],
        description="Marital Status:",
        style={'description_width': 'initial'}
    )
    region_buttons = widgets.ToggleButtons(
        options=["Tier 1", "Tier 2", "Tier 3", "Tier 4"],
        description="Region:",
        style={'description_width': 'initial'}
    )
    num_children_input = widgets.IntText(
        description="Number of Children:",
        value=0,
        style={'description_width': 'initial'}
    )
    submit_button = widgets.Button(description="Calculate", button_style='warning')
    output = widgets.Output()
    marital_status_box = widgets.VBox([])
    region_box = widgets.VBox([])
    children_box = widgets.VBox([])
    submit_box = widgets.VBox([])
    def on_salary_submit(change):
        marital_status_box.children = [marital_status_buttons]
    def on_marital_status_change(change):
        if change.new in ["Married", "Divorced"]:  # Show for both Married and Divorced
            children_box.children = [num_children_input]
        else:
            children_box.children = []  # Hide for Single
        region_box.children = [region_buttons]
    def on_region_selected(change):
        submit_box.children = [submit_button]
    def on_submit_button_click(b):
        display_result(salary_input.value, marital_status_buttons.value, region_buttons.value, num_children_input.value, category_rules, output)
    salary_input.observe(on_salary_submit, names='value')
    marital_status_buttons.observe(on_marital_status_change, names='value')
    region_buttons.observe(on_region_selected, names='value')
    submit_button.on_click(on_submit_button_click)
    display(salary_input, marital_status_box, children_box, region_box, submit_box, output)
main()

FloatText(value=0.0, description='Enter Salary:', style=DescriptionStyle(description_width='initial'))

VBox()

VBox()

VBox()

VBox()

Output()

### FINANCIAL ANALYSIS BY SUBCATEGORY DIVISION (without dummy data)

In [11]:
!pip install tabulate


In [4]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
from tabulate import tabulate
def load_data():
    # Load financial rules
    financial_rules_df = pd.read_excel("Updated_Financial_Analytics.xlsx")
    return financial_rules_df
def calculate_budget(salary, marital_status, region, num_children, category_rules):
    category_row = category_rules[(category_rules['Region'] == region) & (category_rules['Category'] == marital_status)]
    if category_row.empty:
        needs_percentage, wants_percentage, savings_percentage = 50, 30, 20
    else:
        needs_percentage, wants_percentage, savings_percentage = map(lambda x: sum(map(int, x.strip('%').split('-'))) / 2,
                                                                    category_row.iloc[0][['Needs (%)', 'Wants (%)', 'Savings (%)']])
    if marital_status in ["Married", "Divorced"] and num_children:
        needs_percentage += num_children * 5
        wants_percentage -= num_children * 5
    if region == "Tier 1":
        needs_percentage += 5
        wants_percentage -= 5
    elif region == "Tier 4":
        needs_percentage -= 5
        wants_percentage += 5
    needs_amount = salary * (needs_percentage / 100)
    wants_amount = salary * (wants_percentage / 100)
    savings_amount = salary * (savings_percentage / 100)
    return needs_amount, wants_amount, savings_amount
def display_result(salary, marital_status, region, num_children, category_rules, output):
    needs, wants, savings = calculate_budget(salary, marital_status, region, num_children, category_rules)
    budget_df = pd.DataFrame({
        "Category": [marital_status],
        "Region": [region],
        "Salary": [salary],
        "Needs": [needs],
        "Wants": [wants],
        "Savings": [savings]
    })
    needs_breakdown = pd.DataFrame({
        "Category": ["Housing", "Food", "Transportation", "Healthcare", "Debt Payments"],
        "Amount": [needs * 0.4, needs * 0.2, needs * 0.15, needs * 0.1, needs * 0.15]
    })
    wants_breakdown = pd.DataFrame({
        "Category": ["Shopping", "Dining Out", "Entertainment", "Travel", "Hobbies"],
        "Amount": [wants * 0.25, wants * 0.25, wants * 0.2, wants * 0.2, wants * 0.1]
    })
    savings_breakdown = pd.DataFrame({
        "Category": ["Emergency Fund", "Investments", "Retirement Savings", "Large Purchase Goals"],
        "Amount": [savings * 0.3, savings * 0.4, savings * 0.2, savings * 0.1]
    })
    with output:
        output.clear_output()
        print("\n**Budget Plan**")
        print(tabulate(budget_df, headers='keys', tablefmt='grid'))
        print("\n**Needs Breakdown**")
        print(tabulate(needs_breakdown, headers='keys', tablefmt='grid'))
        print("\n**Wants Breakdown**")
        print(tabulate(wants_breakdown, headers='keys', tablefmt='grid'))
        print("\n**Savings Breakdown**")
        print(tabulate(savings_breakdown, headers='keys', tablefmt='grid'))
def main():
    category_rules = load_data()
    salary_input = widgets.FloatText(description="Enter Salary:")
    marital_status_buttons = widgets.ToggleButtons(options=["Single", "Married", "Divorced"], description="Marital Status:")
    region_buttons = widgets.ToggleButtons(options=["Tier 1", "Tier 2", "Tier 3", "Tier 4"], description="Region:")
    num_children_input = widgets.IntText(description="Number of Children:", value=0)
    submit_button = widgets.Button(description="Calculate", button_style='warning')
    output = widgets.Output()
    marital_status_box = widgets.VBox([])
    region_box = widgets.VBox([])
    children_box = widgets.VBox([])
    submit_box = widgets.VBox([])
    def on_salary_submit(change):
        marital_status_box.children = [marital_status_buttons]
    def on_marital_status_change(change):
        children_box.children = [num_children_input] if change.new in ["Married", "Divorced"] else []
        region_box.children = [region_buttons]
    def on_region_selected(change):
        submit_box.children = [submit_button]
    def on_submit_button_click(b):
        display_result(salary_input.value, marital_status_buttons.value, region_buttons.value, num_children_input.value, category_rules, output)
    salary_input.observe(on_salary_submit, names='value')
    marital_status_buttons.observe(on_marital_status_change, names='value')
    region_buttons.observe(on_region_selected, names='value')
    submit_button.on_click(on_submit_button_click)
    display(salary_input, marital_status_box, children_box, region_box, submit_box, output)
main()

FloatText(value=0.0, description='Enter Salary:')

VBox()

VBox()

VBox()

VBox()

Output()

### CLEAR BREAKDOWN OF CATEGORIES

In [3]:
!pip install panel


   ---------------------------------------- 0.0/24.7 MB ? eta -:--:--
   --- ------------------------------------ 2.1/24.7 MB 9.8 MB/s eta 0:00:03
   ------- -------------------------------- 4.5/24.7 MB 10.3 MB/s eta 0:00:02
   ---------- ----------------------------- 6.3/24.7 MB 10.2 MB/s eta 0:00:02
   -------------- ------------------------- 8.7/24.7 MB 10.1 MB/s eta 0:00:02
   ----------------- ---------------------- 11.0/24.7 MB 10.4 MB/s eta 0:00:02
   --------------------- ------------------ 13.4/24.7 MB 10.5 MB/s eta 0:00:02
   ------------------------ --------------- 15.2/24.7 MB 10.4 MB/s eta 0:00:01
   ---------------------------- ----------- 17.6/24.7 MB 10.3 MB/s eta 0:00:01
   -------------------------------- ------- 19.9/24.7 MB 10.4 MB/s eta 0:00:01
   ------------------------------------ --- 22.3/24.7 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------  24.6/24.7 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------- 24.7/24.7 MB 10.2 MB/s

In [9]:
import pandas as pd
import panel as pn

# Load financial rules
def load_data():
    return pd.read_excel("corrected_financial_analytics.xlsx")

# Budget Calculation
def calculate_budget(salary, marital_status, region, num_children, category_rules):
    category_row = category_rules[(category_rules['Region'] == region) & (category_rules['Category'] == marital_status)]
    if category_row.empty:
        needs_percentage, wants_percentage, savings_percentage = 50, 30, 20
    else:
        needs_percentage, wants_percentage, savings_percentage = map(lambda x: sum(map(int, x.strip('%').split('-'))) / 2,
                                                                    category_row.iloc[0][['Needs (%)', 'Wants (%)', 'Savings (%)']])
    if marital_status in ["Married", "Divorced"] and num_children:
        needs_percentage += num_children * 5
        wants_percentage -= num_children * 5
    needs_amount = salary * (needs_percentage / 100)
    wants_amount = salary * (wants_percentage / 100)
    savings_amount = salary * (savings_percentage / 100)
    return needs_amount, wants_amount, savings_amount

# UI Components
category_rules = load_data()
pn.extension()

salary_input = pn.widgets.FloatInput(name="Salary (₹)")
marital_status_input = pn.widgets.RadioButtonGroup(name="Marital Status", options=["Single", "Married", "Divorced"])
region_input = pn.widgets.RadioButtonGroup(name="Region", options=["Metro", "Urban", "Semi-Urban", "Rural"])
num_children_input = pn.widgets.IntInput(name="Number of Children", value=0)
house_type_input = pn.widgets.Select(name="House Type", options=["Own", "Rent"])
house_expense_input = pn.widgets.FloatInput(name="Total House Expenses (₹)", visible=False)
rent_amount_input = pn.widgets.FloatInput(name="Rent Amount (₹)", visible=False)

# Loans Section
loans_question = pn.pane.Markdown("### Do you have Loans?")
has_loans_input = pn.widgets.RadioButtonGroup(name="Do you have Loans?", options=["Yes", "No"])
num_loans_input = pn.widgets.IntInput(name="Number of Loans", value=0, visible=False)
loan_inputs = pn.Column()

transport_expense_input = pn.widgets.FloatInput(name="Transport Expense (₹)")
food_expense_input = pn.widgets.FloatInput(name="Food Expense (₹)")
medical_expense_input = pn.widgets.FloatInput(name="Medical Expense (₹)")

calculate_button = pn.widgets.Button(name="Calculate Budget", button_type='primary')
output_area = pn.pane.Markdown()
output_area_table = pn.Column()

def update_loan_widgets(event):
    loan_inputs.clear()
    if has_loans_input.value == "Yes":
        num_loans_input.visible = True
        for i in range(num_loans_input.value):
            loan_inputs.append(pn.widgets.FloatInput(name=f"Loan {i+1} Amount (₹, Monthly)"))
            loan_inputs.append(pn.widgets.IntInput(name=f"Loan {i+1} Tenure (Years)"))
    else:
        num_loans_input.visible = False

def update_house_expense(event):
    if house_type_input.value == "Own":
        house_expense_input.visible = True
        house_expense_input.name = "Total House Expenses (₹)"
        rent_amount_input.visible = False
    elif house_type_input.value == "Rent":
        rent_amount_input.visible = True
        rent_amount_input.name = "Rent Amount (₹)"
        house_expense_input.visible = False

has_loans_input.param.watch(update_loan_widgets, 'value')
num_loans_input.param.watch(update_loan_widgets, 'value')
house_type_input.param.watch(update_house_expense, 'value')

def on_calculate(event):
    # Calculate budget amounts
    needs, wants, savings = calculate_budget(
        salary_input.value, marital_status_input.value, region_input.value, num_children_input.value, category_rules
    )

    # Define financial breakdowns
    savings_breakdown = pd.DataFrame({
        "Category": ["Short-Term Savings", "Long-Term Savings"],
        "Amount (₹)": [savings * 0.4, savings * 0.6]
    })
    
    short_term_savings = pd.DataFrame({
        "Sub-Category": ["Down Payment", "Emergency Fund", "Education"],
        "Amount (₹)": [savings * 0.15, savings * 0.15, savings * 0.1]
    })
    
    long_term_savings = pd.DataFrame({
        "Sub-Category": ["Retirement Fund", "Life Insurance", "Children Higher Education", "Wealth Creation Investments"],
        "Amount (₹)": [savings * 0.2, savings * 0.1, savings * 0.15, savings * 0.15]
    })
    
    # Define Needs Breakdown with Healthcare Sub-Categories
    needs_breakdown_data = {
        "Sub-Category": ["Housing", "Utilities", "Groceries", "Transportation"],
        "Amount (₹)": [needs * 0.3, needs * 0.2, needs * 0.2, needs * 0.2]
    }

    # Add Healthcare breakdown (sub-categories)
    healthcare_total = needs * 0.1
    needs_breakdown_data["Sub-Category"].append("Healthcare")
    needs_breakdown_data["Amount (₹)"].append(healthcare_total)

    healthcare_subcategories = {
        "Sub-Category": ["Medical Insurance", "Doctor Visits", "Medications", "Emergency Fund"],
        "Amount (₹)": [healthcare_total * 0.4, healthcare_total * 0.3, healthcare_total * 0.2, healthcare_total * 0.1]
    }

    # Add Healthcare Subcategories to the needs breakdown
    needs_breakdown_data["Sub-Category"].extend(healthcare_subcategories["Sub-Category"])
    needs_breakdown_data["Amount (₹)"].extend(healthcare_subcategories["Amount (₹)"])

    # Add Bills section if House Type is "Own"
    if house_type_input.value == "Own":
        needs_breakdown_data["Sub-Category"].append("Bills")
        needs_breakdown_data["Amount (₹)"].append(house_expense_input.value if house_expense_input.value else 0)

    needs_breakdown = pd.DataFrame(needs_breakdown_data)

    wants_breakdown = pd.DataFrame({
        "Sub-Category": ["Dining Out", "Entertainment", "Travel", "Shopping"],
        "Amount (₹)": [wants * 0.25, wants * 0.25, wants * 0.25, wants * 0.25]
    })

    # Summary text
    summary_text = f"""
    ## Budget Plan
    **Marital Status:** {marital_status_input.value}  
    **Region:** {region_input.value}  
    **Salary:** ₹{salary_input.value:,.2f}  

    **Needs Amount:** ₹{needs:,.2f}  
    **Wants Amount:** ₹{wants:,.2f}  
    **Savings Amount:** ₹{savings:,.2f}  
    """

    output_area.object = summary_text  # Display textual summary

    # Display tables properly
    output_area_table.objects = [
        pn.pane.Markdown("### Needs Breakdown"), pn.pane.DataFrame(needs_breakdown, width=500),
        pn.pane.Markdown("### Wants Breakdown"), pn.pane.DataFrame(wants_breakdown, width=500),
        pn.pane.Markdown("### Savings Breakdown"), pn.pane.DataFrame(savings_breakdown, width=500),
        pn.pane.Markdown("### Short-Term Savings Breakdown"), pn.pane.DataFrame(short_term_savings, width=500),
        pn.pane.Markdown("### Long-Term Savings Breakdown"), pn.pane.DataFrame(long_term_savings, width=500),
    ]

calculate_button.on_click(on_calculate)

# Dashboard Layout
dashboard = pn.Column(
    "## Financial Budget Planner",
    salary_input, marital_status_input, region_input, num_children_input,
    house_type_input, house_expense_input, rent_amount_input,
    loans_question, has_loans_input, num_loans_input, loan_inputs,
    transport_expense_input, food_expense_input, medical_expense_input,
    calculate_button, output_area, output_area_table  # Text summary + tables
)

dashboard.servable()


Column
    [0] Markdown(str)
    [1] FloatInput(name='Salary (₹)')
    [2] RadioButtonGroup(name='Marital Status', options=['Single', 'Married', ...], value='Single')
    [3] RadioButtonGroup(name='Region', options=['Metro', 'Urban', ...], value='Metro')
    [4] IntInput(name='Number of Children')
    [5] Select(name='House Type', options=['Own', 'Rent'], value='Own')
    [6] FloatInput(name='Total House E..., visible=False)
    [7] FloatInput(name='Rent Amount (₹)', visible=False)
    [8] Markdown(str)
    [9] RadioButtonGroup(name='Do you have Loans?', options=['Yes', 'No'], value='Yes')
    [10] IntInput(name='Number of Loans', visible=False)
    [11] Column()
    [12] FloatInput(name='Transport Expense (₹)')
    [13] FloatInput(name='Food Expense (₹)')
    [14] FloatInput(name='Medical Expense (₹)')
    [15] Button(button_type='primary', name='Calculate Budget')
    [16] Markdown(None)
    [17] Column()